In [2]:
import numpy as np
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import ResNet50

SEED = 42
import random 
random.seed(SEED)
import numpy as np
np.random.seed(SEED)
import tensorflow as tf
tf.random.set_seed(SEED)

# ** Inputs **

In [3]:
# Choose which image folder name you want to use.
image_folder_name = 'FMD_tvt'

#image size
img_size = (150, 150)

#Batch Size of Data
batch_size = 32


Model_Test_Name = 'MobilNet.h5'

# Setup of Paths and Values

In [4]:
tvt_folders = ['train', 'val', 'test']
im_folders = ['image', 'mask']

Fmd_class_path = os.path.join(image_folder_name, tvt_folders[0], im_folders[0])
classes = os.listdir(os.path.join(Fmd_class_path))
print(classes)

['fabric', 'foliage', 'glass', 'leather', 'metal', 'paper', 'plastic', 'stone', 'water', 'wood']


In [5]:
image_path_test = os.path.join(image_folder_name, tvt_folders[2], im_folders[0])
mask_path_test = os.path.join(image_folder_name, tvt_folders[2], im_folders[1])

# Load in Data and Make Datasets

Define Three functions

In [6]:
def fun_load_image(x):
    byte_img = tf.io.read_file(x) #Tensorflow way of loading in image
    img = tf.io.decode_jpeg(byte_img)  #Need both lines of code
    return img

In [7]:
def make_dataset_imgs(tvt_folder, classes, img_size):
    ds = tf.data.Dataset.list_files(os.path.join(image_folder_name, tvt_folder, 'image', classes[0], "*.jpg"), shuffle=False)
    for i in range(len(classes)-1):
        ds_class = tf.data.Dataset.list_files(os.path.join(image_folder_name, tvt_folder, 'image', classes[i+1], "*.jpg"), shuffle=False)
        ds = ds.concatenate(ds_class)

    ds = ds.map(fun_load_image)
    ds = ds.map(lambda x: tf.image.resize(x, img_size))
    ds = ds.map(lambda x: x/255)
    return ds


In [8]:
def make_dataset_labels(image_path):
    class_labels = []
    for class_ in classes:
        class_val = classes.index(class_)
        num_in_class = len(os.listdir(os.path.join(image_path, class_)))
        class_labels_ = [class_val]*num_in_class
        class_labels = class_labels + class_labels_

    label_ds = tf.data.Dataset.from_tensor_slices(class_labels)
    return label_ds

Test Data

In [9]:
test_ds = make_dataset_imgs(tvt_folders[2], classes, img_size)
test_labels_ds = make_dataset_labels(image_path_test)

Zipping and Shuffle

In [10]:
test = tf.data.Dataset.zip((test_ds, test_labels_ds))
test = test.shuffle(10000)

Batch and Prefetch

In [11]:
test = test.cache().batch(batch_size).prefetch(buffer_size=10)

# MODEL TESTING

In [12]:
Model_path = os.path.join('Models', Model_Test_Name)

model = tf.keras.models.load_model(Model_path)

In [13]:
X_test = test.map(lambda a, b: a)
y_test = test.map(lambda a, b: b)

In [14]:
predictions = model.predict(X_test)

7/7 [==============================] - 2s 229ms/step


In [15]:
class_preds = []
for lst in predictions:
    # print(max(lst))
    class_preds.append(np.where(lst == max(lst))[0][0])
    
y_test_true = []
for elem in y_test:
    for pred in elem.numpy():
        y_test_true.append(pred)


# Final Test Accuracy

In [16]:
comparison = []
for i in range(len(y_test_true)):
    # print(y_test_true[i], class_preds[i])
    if y_test_true[i] == class_preds[i]:
        comparison.append(1)
    if y_test_true[i] != class_preds[i]:
        comparison.append(0)

print(f'Final Test Accuracy = {sum(comparison)/len(comparison)}')

200
129
class_accuracy = 0.645
